In [1]:
from sklearn.linear_model import LogisticRegression
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pwd

/content


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/assign03

/content/drive/MyDrive/Colab Notebooks/assign03


In [4]:
import numpy as np 
import matplotlib.pyplot as plt

# install
## numpy
## matplotlib

#### 손 글씨로 만들어진 0~9까지의 숫자 이미지 데이터셋

### data load & preprocessing

In [5]:
from dataset.mnist import load_mnist

(train_raw_img, train_label), (test_raw_img, test_label) = load_mnist(flatten=False, normalize=False)
print(train_raw_img.shape)
print(train_label.shape)

(60000, 1, 28, 28)
(60000,)


In [6]:
np.unique(train_label, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949]))

In [7]:
# train_dataset split according to the number

new_train_img = [[] for _ in range(10)]
new_train_label = [[] for _ in range(10)]

for i in range(len(train_label)) :
    new_train_img[train_label[i]].append(train_raw_img[i])
    new_train_label[train_label[i]].append(train_label[i])

# print(len(new_train_img[0])) # 0에 해당하는 image 개수
# print(new_train_img[0][0].shape) # 0에 해당하는 image중 첫번째 image의 shape

### 1. Create a classifier that distinguishes between zero and non-zero (using logistic regression)

##### 데이터셋에서 주어진 이미지가 0인지 아닌지 구별하는 이진 분류를 하기 위한 Logistic Regression(로지스틱 회귀)를 작성하라

In [8]:
# Target Number(idx)를 입력받아 Positive와 Negative Sample의 비율을 맞춰 학습 데이터셋 반환
def make_sample(idx) :
    sample_img = []
    sample_label = []
    
    # data sampling 
    for i in range(10) :
        if i == idx :
            sample_img += new_train_img[i][:1000]
            sample_label += (new_train_label[i][:1000])
        else :
            sample_img += new_train_img[i][:111]
            sample_label += (new_train_label[i][:111])

    sample_img = np.array(sample_img)
    sample_label = np.array(sample_label)
    
    # normalization (set value 0 ~ 1)
    sample_img = sample_img.astype('float')/255
    
    # target number는 1, 아니면 0
    sample_label = np.where(sample_label==idx, 1 ,0)
    
    # reshape
    sample_img = sample_img.reshape(len(sample_img.squeeze()), -1)
    sample_label = sample_label.reshape(len(sample_label.squeeze()), -1)
    
    return sample_img, sample_label

In [9]:
train_X, train_y = make_sample(idx = 0) # idx = target number
train_X = np.insert(train_X, 0, 1, axis=1) # bias 추가

# print(train_y.shape)

In [10]:
# cross entropy loss
def CrossEntropyLoss(preds, y) :
    delta = 1e-7
    loss = np.sum(-y*np.log(preds+delta)-(1-y)*np.log(1-preds+delta))/len(preds)
        
    return loss

loss = CrossEntropyLoss(np.zeros((len(train_X), 1)), train_y)
print('loss : ', loss)

loss :  8.063079315186753


In [11]:
# Train 이미지와 Label을 입력받아 학습을 진행, 학습된 Parameter 반환

def train(X, y) :
    """_summary_

    Args:
        X : train_X
        y : train_y

    Returns:
        w : weight
    """
    w = np.random.randn(len(X[0]), 1) # weight initialization
    lr = 0.1 # learning rate(수정 가능)
    step = 0
    acc = 0
    
    while (acc <= 0.9) :  # 좀 더 나은 정확성을 위해 acc이이 0.9 이상일 때 반복문 탈출
        step += 1
        
        # predict
        preds = 1 / (1 + np.exp(-np.dot(X, w)))
        loss = CrossEntropyLoss(preds, y)
        
        result = np.where(preds>0.5, 1, 0)
        acc = np.sum(np.where(result==y, True, False))/len(preds)
        
        print("total step : %d " % step)
        print("error : %f, accuarcy : %f" % (loss, acc))
        
        # gradient descent
        gradient = np.dot(X.T, (preds - y)) / len(preds)
        w -= lr * gradient
        
    return w

In [12]:
# save weight

test_w = train(train_X, train_y)

total step : 1 
error : 5.216721, accuarcy : 0.524762
total step : 2 
error : 4.768590, accuarcy : 0.535268
total step : 3 
error : 4.336914, accuarcy : 0.542271
total step : 4 
error : 3.941176, accuarcy : 0.544772
total step : 5 
error : 3.593916, accuarcy : 0.548274
total step : 6 
error : 3.301579, accuarcy : 0.567284
total step : 7 
error : 3.060888, accuarcy : 0.571786
total step : 8 
error : 2.862744, accuarcy : 0.579290
total step : 9 
error : 2.698728, accuarcy : 0.591796
total step : 10 
error : 2.561594, accuarcy : 0.603302
total step : 11 
error : 2.445090, accuarcy : 0.608804
total step : 12 
error : 2.344139, accuarcy : 0.613807
total step : 13 
error : 2.254881, accuarcy : 0.621811
total step : 14 
error : 2.174511, accuarcy : 0.624812
total step : 15 
error : 2.101057, accuarcy : 0.626313
total step : 16 
error : 2.033149, accuarcy : 0.634317
total step : 17 
error : 1.969815, accuarcy : 0.643322
total step : 18 
error : 1.910347, accuarcy : 0.651326
total step : 19 
er

In [13]:
# Target Number(idx)와 학습된 Parameter를 입력받아 Accuracy 계산

def eval(idx, w) :
    """_summary_

    Args:
        idx : target_number
        w : parameter
    """
    test_X = test_raw_img.astype('float')/255    
    test_X = test_X.reshape(len(test_X.squeeze()), -1)
    test_X = np.insert(test_X, 0, 1, axis=1) # + bias

    test_y = np.where(test_label==idx, 1 ,0)
    test_y = test_y.reshape(len(test_y.squeeze()), -1)
    
    preds = 1/(1+np.exp(-test_X.dot(w)))
    result = np.where(preds>0.5, 1, 0)
    
    acc = np.sum(np.where(result==test_y, True, False))/len(preds)
    print('accuracy : ', acc)

eval(idx=0, w=test_w)

accuracy :  0.8924


### 2. multi class single label classification (using logistic regression)

##### 데이터셋을 Logistic Regression(로지스틱 회귀)를 이용하여 N개의 멀티 분류 모델을 활용해 주어진 이미지의 숫자를 예측

In [14]:
# train several binary linear regression classifier (0~9)
classifiers = []
for i in range(10) :
  train_X, train_y = make_sample(i) # i = target number
  train_X = np.insert(train_X, 0, 1, axis=1) # bias 추가
  clf = LogisticRegression(max_iter=5000) # Logistic Regression 이용용
  clf.fit(train_X, train_y.ravel())
  classifiers.append(clf)

In [15]:
# eval

test_X = test_raw_img.astype('float')/255    
test_X = test_X.reshape(len(test_X.squeeze()), -1)
test_X = np.insert(test_X, 0, 1, axis=1) # + bias
print('shape of test_X : ', test_X.shape)
print('shape of test_label : ', test_label.shape)

probabilities = []
for clf in classifiers:
  proba = clf.predict_proba(test_X)[:, 1]
  probabilities.append(proba)

probabilities = np.array(probabilities).T

# make prediction using argmax
max_pred = np.argmax(probabilities, axis=1) # shape : (10000,1) or (10000,)

acc = np.sum(np.where(test_label==max_pred, True, False))/len(test_X)
print('accuracy : ', acc)

shape of test_X :  (10000, 785)
shape of test_label :  (10000,)
accuracy :  0.8824


### L2 Regularization

##### Overfitting 방지를 위한 L2 Regularization 구현

In [18]:
def L2_regularization(w, lamb) :  # L2 Regularization 적용한 w 반환
    return lamb * np.sum(w**2)

def CrossEntropyLoss_L2(preds, y, w, lamb) :
    delta = 1e-7
    loss = np.sum(-y*np.log(preds+delta)-(1-y)*np.log(1-preds+delta))/len(preds) + L2_regularization(w, lamb) # 기존 loss의 L2 정규화 적용용
    return loss

def train_L2(X, y, lamb) :
    w = np.random.randn(len(X[0]), 1) # weight initialization
    lr = 0.1
    step = 0
    acc = 0

    while (acc <= 0.9) :  # 좀 더 나은 정확성을 위해 acc이이 0.9 이상일 때 반복문 탈출
        step += 1
        
        # predict
        preds = 1 / (1 + np.exp(-np.dot(X, w)))
        loss = CrossEntropyLoss_L2(preds, y, w, lamb) # L2 정규화 적용된 loss를 받는다.
        
        result = np.where(preds>0.5, 1, 0)
        acc = np.sum(np.where(result==y, True, False))/len(preds)
        
       
        
        # gradient descent
        gradient = np.dot(X.T, (preds - y)) / len(preds) + 2 * lamb * w
        w -= lr * gradient

    # 마지막 값만 도출출
    print("\nL2_total step : %d " % step)
    print("L2_error : %f, L2_accuarcy : %f" % (loss, acc))

    return w

org_w = train(train_X, train_y)
l2_w = train_L2(train_X, train_y, 0.01)

org_loss = CrossEntropyLoss(np.zeros((len(train_X), 1)), train_y)
print('loss : ', org_loss)

l2_loss =  CrossEntropyLoss_L2(np.zeros((len(train_X), 1)), train_y, l2_w, 0.01)
print('L2_loss : ', l2_loss)

total step : 1 
error : 6.467850, accuarcy : 0.491246
total step : 2 
error : 5.836976, accuarcy : 0.487744
total step : 3 
error : 5.097173, accuarcy : 0.490745
total step : 4 
error : 4.299391, accuarcy : 0.493747
total step : 5 
error : 3.529770, accuarcy : 0.493747
total step : 6 
error : 2.879032, accuarcy : 0.511756
total step : 7 
error : 2.401287, accuarcy : 0.536768
total step : 8 
error : 2.089865, accuarcy : 0.562781
total step : 9 
error : 1.897851, accuarcy : 0.579290
total step : 10 
error : 1.776583, accuarcy : 0.597799
total step : 11 
error : 1.694471, accuarcy : 0.606803
total step : 12 
error : 1.634014, accuarcy : 0.617809
total step : 13 
error : 1.585846, accuarcy : 0.628814
total step : 14 
error : 1.544969, accuarcy : 0.637319
total step : 15 
error : 1.508699, accuarcy : 0.642321
total step : 16 
error : 1.475578, accuarcy : 0.647324
total step : 17 
error : 1.444791, accuarcy : 0.654327
total step : 18 
error : 1.415870, accuarcy : 0.659330
total step : 19 
er

L2 Regularization이 적용된 후 는 적용 전에 비해 Accuracy에 도달하는 과정에서 error와 loss가 크게 나오지만, total step은 적게 나온다.

이러한 이유로는 L2 Regularization을 적용하면 손실 함수에 정규화 항이 추가되어 손실 값이 커져 가중치의 크기를 제한하는 역할을 하기 때문이다.

lamb 값 역시 중요하다. lamb 값이 너무 크다면 모델이 데이터에 충분히 적합하지 못하게 되고, 너무 작다면 정규화의 효과를 보지 못한다